In [1]:
!pip install streamlit pyngrok transformers sumy -q
!pip install langchain_google_genai langchain_core -q
!pip install PyMuPDF -q
!pip install python-docx -q
!pip install PyPDF2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.1 MB/s eta 0:00:00


In [2]:
!pip install fpdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.8/227.8 kB 14.7 MB/s eta 0:00:00


In [3]:
!pip install -U  langchain langchain-community pypdf langchain-google-genai langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00


In [12]:
app_code = '''
import streamlit as st
from transformers import BartForConditionalGeneration, BartTokenizer, T5ForConditionalGeneration, T5Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from langchain.document_loaders import PyPDFLoader
from fpdf import FPDF
from PyPDF2 import PdfReader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

nltk.download('punkt')
nltk.download('stopwords')
import nltk
nltk.download('punkt_tab')


st.set_page_config(layout="wide")


def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    # Split the documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    return docs


def count_words_in_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader =PdfReader(pdf_file)
            text = ""

            for page in reader.pages:
                text += page.extract_text()
            words = text.split()
            return len(words)
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return 0

def pdf_to_text(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PdfReader(pdf_file)
            for page in reader.pages:
                text += page.extract_text()

    except Exception as e:
        text = f"An error occurred: {e}"
    return text


def text_to_pdf(text, pdf_path):
    # Create an instance of FPDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Split text into lines and add them to the PDF
    for line in text.split(". "):
        pdf.cell(200, 10, txt=line, ln=True, align='L')

    # Save the PDF to the specified path
    pdf.output(pdf_path)


def load_llm(model="gemini-1.5-flash"):
    llm = ChatGoogleGenerativeAI(
        model=model,
        temperature=0,
        max_tokens=1024,
        timeout=None,
        max_retries=2
    )
    return llm

def get_prompt_template(num_words):
    return ChatPromptTemplate.from_messages(
        [
            ("system", f"Write a concise summary of the following in {num_words} words:  "),
            ("human", "{context}")
        ]
    )



def get_chain(num_words, model_type, llm):
    if model_type == "refine":
        base_prompt = PromptTemplate(
            input_variables=["text"],
            template=f"""
            Summarize the following text in {num_words} words or  more:
            {{text}}
            Summary:"""
        )

        refine_prompt = PromptTemplate(
            input_variables=["existing_summary", "text"],
            template=f"""
            Your previous summary is:
            {{existing_summary}}
            Refine it to ensure it remains concise and exactly {num_words} words or more.
            Updated Summary:"""
        )
        chain = load_summarize_chain(llm, chain_type="refine", verbose=True, question_prompt=base_prompt, refine_prompt=refine_prompt)
        return chain

    elif model_type == "map_reduce":
        map_prompt = PromptTemplate(
            input_variables=["text"],
            template=f"""
            Summarize the following text in {num_words} words :
            {{text}}
            Summary:"""
        )

        combine_prompt = PromptTemplate(
            input_variables=["context", "text"],
            template=f"""
            The following is the current summary:
            {{context}}

            Refine the summary based on the additional text below. Ensure the final summary is concise
            and exactly {num_words} words:
            {{text}}
            Updated Summary:"""
        )

        # Create the Map-Reduce chain
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            verbose=True,
            map_prompt=map_prompt,
            combine_prompt=combine_prompt
        )
        return chain

    else:
        raise ValueError(f"Unsupported model_type: {model_type}")



def summarize_text_llm(model_type, num_words, inputs):
    llm = load_llm("gemini-1.5-flash")

    if model_type == "stuffed_document":
        prompt = get_prompt_template(num_words)
        chain = prompt | llm
        result = chain.invoke({
            "context": inputs,
            "num_words": num_words
        })
        return result.content

    elif model_type =="refine":
        loader = PyPDFLoader(inputs)
        docs = loader.load_and_split()
        chain=get_chain(num_words,model_type,llm)
        summary = chain.invoke(docs)
        return summary['output_text']

    elif model_type =="map_reduce":
        # Load and split documents
        docs = load_and_split_pdf(inputs)

        # Get the chain
        chain = get_chain(num_words, model_type, llm)

        # Invoke the chain with the documents
        summary = chain.invoke({
    "input_documents": docs,
    "context": ""  # Initializing context with an empty string
})
        return summary['output_text']

def load_abstractive_models():
    bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')
    t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
    return (bart_model, bart_tokenizer), (t5_model, t5_tokenizer)


def abstractive_summary(text, model_type, num_words):
    if model_type == "BART":
        model, tokenizer = load_abstractive_models()[0]
    elif model_type == "T5":
        model, tokenizer = load_abstractive_models()[1]

    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'], max_length=int(num_words * 1.1), min_length=int(num_words *1),
        length_penalty=0.05, num_beams=4, early_stopping=False
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


def extractive_summary(text, method, num_of_line=2):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    if method == "TextRank":
        summarizer = TextRankSummarizer()
        summary = summarizer(parser.document, num_of_line)
        return ' '.join(str(sentence) for sentence in summary)
    elif method == "LSA":
        summarizer = LsaSummarizer()
        summary = summarizer(parser.document, num_of_line)
        return ' '.join(str(sentence) for sentence in summary)
    elif method == "Frequency":
        return frequency_summary(text, num_of_line)


def frequency_summary(text, num_of_line):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text.lower())
    freq_table = defaultdict(int)
    for word in words:
        if word.isalpha() and word not in stop_words:
            freq_table[word] += 1

    sentences = sent_tokenize(text)
    sentence_scores = defaultdict(int)
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in freq_table:
                sentence_scores[sentence] += freq_table[word]

    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    summary = summary_sentences[:num_of_line]
    return ' '.join(summary)





def main():
    st.title("Text Summarization App")
    st.write("Choose a summarization method:")

    incol, outcol = st.columns(2)

    with incol:
        intype,method,algorithm = st.columns(3)
        with intype:
            input_type = st.radio("Select Input Type", ["Text", "Document"])
        with method:
            summarization_type = st.radio("Select Summarization Method", ["Extractive", "Abstractive", "LLM"])
        with algorithm:
            if summarization_type == "Extractive":
                method = st.radio("Select Algorithm", ["TextRank", "LSA", "Frequency"])
            elif summarization_type == "Abstractive":
                model_type = st.radio("Select Algorithm", ["BART", "T5"])
            elif summarization_type == "LLM":
                model_type = st.radio("Select Algorithm", ["stuffed_document", "refine", "map_reduce"])
        if summarization_type == "Extractive":
            num_of_lines = st.number_input("Number of lines in summary:", min_value=1, value=2)
        else:
            num_words = st.number_input("Number of words in summary:", min_value=10, value=100)

        if input_type == "Text":
            text_input = st.text_area("Enter text to summarize:", height=200)
        elif input_type == "Document":
            pdf_file = st.file_uploader("Upload a PDF file", type=["pdf"])
            if pdf_file:
                # Save the uploaded file to a temporary path
                with open("temp_uploaded_file.pdf", "wb") as f:
                    f.write(pdf_file.read())
                pdf_file_path = "temp_uploaded_file.pdf"
            else:
                st.warning("Please upload a PDF file.")
                pdf_file_path = None

    if st.button("Summarize"):
        if input_type =="Text":
            in_word= len(text_input.split(" "))
        elif input_type == "Document":
            in_word=count_words_in_pdf(pdf_file_path)

        if summarization_type == "Abstractive":
            if input_type=="Document":
              text_input=pdf_to_text(pdf_file_path)
            summary = abstractive_summary(text_input, model_type, num_words)
        elif summarization_type == "Extractive":
            if input_type=="Document":
              text_input=pdf_to_text(pdf_file_path)
            summary = extractive_summary(text_input, method, num_of_lines)
        elif summarization_type == "LLM":
            if model_type == "stuffed_document" and input_type=="Document":
                text_input=pdf_to_text(pdf_file_path)
            elif model_type in ["refine", "map_reduce"] :
                if input_type=="Text":
                    with open("temp_uploaded_file.pdf", "wb") as f:
                        text_to_pdf(text_input, "temp_uploaded_file.pdf")
                    pdf_file_path = "temp_uploaded_file.pdf"
                text_input=pdf_file_path
            summary = summarize_text_llm(model_type, num_words, text_input)
        with outcol:
            st.subheader("Summary")
            st.write(summary)
            out_word=len(summary.split(" "))
            out_percent=round((out_word*100/in_word),2)
            st.write(f'The text was reduced from {in_word} words to {out_word} words.({out_percent}%)')


if __name__ == "__main__":
    main()'''

with open('text_summarization.py', 'w') as f:
    f.write(app_code)


In [5]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')
os.environ['ngrok_token']=userdata.get('my_auth_token')

In [6]:
!ngrok authtoken $ngrok_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print(f"Streamlit App is live at: {public_url}")


Streamlit App is live at: NgrokTunnel: "https://bc35-35-199-53-189.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run text_summarization.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.199.53.189:8501

